# Join Adjacency Matrix, Gene Expression & Labels

In [7]:
import pandas
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import h5py

## Get Gene Expression

In [8]:
# params
TIMES_STD_THRESHOLD = 10.
MAX_ZEROS_ALLOWED = .5

In [9]:
# load data
gene_expression = pandas.DataFrame.from_csv('gene_expression/normalized-counts-labels.csv',
                                            encoding='utf-8', sep=',')
ge_nonames = gene_expression.drop('Name', axis=1)

# kick out super highly expressed genes
threshold = ge_nonames.mean(axis=1).std()*TIMES_STD_THRESHOLD
#print ("Threshold Gene Expression: {}".format(threshold))
anomalies = gene_expression[ge_nonames.mean(axis=1) > threshold]
ge_anomalies_removed = ge_nonames.drop(anomalies.index)

# kick out genes with too many zeros
ge_zeros_removed = ge_anomalies_removed[ge_anomalies_removed.astype('bool').mean(axis=1)>=(1-MAX_ZEROS_ALLOWED)]

# scaling
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(ge_zeros_removed)
ge_scaled = pandas.DataFrame(scaled_features,
                             index=ge_zeros_removed.index,
                             columns=ge_zeros_removed.columns)

# print some information
print ("Had gene expression for {} genes in the beginning.".format(ge_nonames.shape[0]))
print ("kicked out {} super highly expressed genes".format(ge_nonames.shape[0] - ge_anomalies_removed.shape[0]))
print ("Kicked out {} genes with more than {}% zeros".format(ge_anomalies_removed.shape[0]-ge_zeros_removed.shape[0],
                                                             MAX_ZEROS_ALLOWED*100.
                                                            ))
print ("==> Left with gene expression for {} genes".format(ge_scaled.shape[0]))

Had gene expression for 41424 genes in the beginning.
kicked out 9 super highly expressed genes
Kicked out 15021 genes with more than 50.0% zeros
==> Left with gene expression for 26394 genes


## Get PPI Network according to Gene Expression Data

In [35]:
gene_names = gene_expression.loc[ge_scaled.index].Name.values
gene_ids = ge_scaled.index
gene_expression

,Name,Pam3T16,Pam3T8,Pam3T16.1,Pam3T8.1,Pam3T16.2,Pam3T8.2,ControlT8,ControlT16,ControlT8.1,...,gfpmT8.1,gfpmT16.1,gfpmT8.2,gfpmT16.2,gfppT8,gfppT16,gfppT8.1,gfppT16.1,gfppT8.2,gfppT16.2
Ensembl-ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,TSPAN6,6.736327,8.519666,10.773706,10.813725,2.427830,7.425264,9.417385,7.920964,10.812953,...,12.529836,6.769370,13.440488,5.248289,14.583375,13.538040,18.220493,14.908853,16.156797,30.745221
ENSG00000000419,DPM1,97.676739,73.025712,132.362676,110.840686,87.401897,87.865624,92.725018,105.132797,121.218890,...,127.983328,147.421826,104.163784,86.596770,298.959192,239.172041,268.101547,155.611158,283.898001,247.298513
ENSG00000000457,SCYL3,84.204085,74.242807,61.564035,68.486928,78.499852,92.815800,86.929705,72.728852,66.585024,...,76.074006,63.180782,117.604272,94.469203,83.125239,79.724014,76.786365,62.430824,85.400212,93.572410
ENSG00000000460,C1orf112,61.468982,48.683808,70.798641,63.080065,63.123592,76.727728,68.819350,68.408327,77.397977,...,56.384263,72.958761,81.482960,51.608176,24.791738,33.092987,29.933668,59.635414,30.005480,26.734974
ENSG00000000938,FGR,53.890614,150.919804,143.136382,362.259802,156.999704,160.880721,62.299622,61.207450,189.511222,...,333.830638,257.988195,186.486774,70.851902,129.792040,97.774734,272.005939,245.996082,140.794944,101.592903
ENSG00000000971,CFH,18.524899,21.907714,43.094825,53.167483,29.943242,23.513336,4.346485,2.880351,15.365775,...,11.634848,17.299500,6.720244,6.997719,5.833350,7.521133,5.205855,11.181640,0.000000,1.336749
ENSG00000001036,FUCA2,90.098371,59.637665,84.650549,82.905228,84.164790,89.103168,119.528344,123.134988,144.552104,...,107.398597,136.139543,103.323753,80.473766,141.458740,118.833907,96.308323,109.952794,114.251635,98.919405
ENSG00000001084,GCLC,317.449401,109.538568,169.301098,410.020423,384.406491,128.704577,262.237943,301.716724,393.250014,...,367.840193,368.554564,218.407934,289.530614,457.917983,544.530055,183.506399,322.403956,459.314652,493.260277
ENSG00000001167,NFYA,225.666948,178.912994,155.449190,196.449345,191.798607,198.007041,163.717611,126.015338,156.503262,...,139.618175,173.747152,177.246439,165.321106,354.376018,427.200375,428.181597,230.155425,371.606327,442.463826


In [11]:
# read interaction data
interactions = pandas.DataFrame.from_csv('ppi_network/ConsensusPathDB_human_PPI.csv',
                                         header=1,
                                         sep='\t',
                                         encoding='utf8'
                                        )
interactions_nona = interactions.dropna()

# construct Adjacency Matrix
N = len(gene_names)
print (N)
adjacency_matrix = np.zeros((N, N), np.uint8)
adj_df = pandas.DataFrame(adjacency_matrix, index=gene_names, columns=gene_names)

count = 0
for index, row in interactions_nona.iterrows():
    if row.interaction_confidence > 0.5:
        interaction_partners = row.interaction_participants.split(',')
        if len(interaction_partners) == 2:
            i1 = interaction_partners[0].split('_')[0].strip() # get rid of "_HUMAN" at end of prot name
            i2 = interaction_partners[1].split('_')[0].strip()
            if i1 in gene_names and i2 in gene_names:
                adj_df.ix[i1, i2] = 1
                adj_df.ix[i2, i1] = 1
            
    count += 1
    if count % 10000 == 0:
        print ("Processed {} out of {} rows".format(count, interactions_nona.shape[0]))

26394
Processed 10000 out of 272744 rows
Processed 20000 out of 272744 rows
Processed 30000 out of 272744 rows
Processed 40000 out of 272744 rows
Processed 50000 out of 272744 rows
Processed 60000 out of 272744 rows
Processed 70000 out of 272744 rows
Processed 80000 out of 272744 rows
Processed 90000 out of 272744 rows
Processed 100000 out of 272744 rows
Processed 110000 out of 272744 rows
Processed 120000 out of 272744 rows
Processed 130000 out of 272744 rows
Processed 140000 out of 272744 rows
Processed 150000 out of 272744 rows
Processed 160000 out of 272744 rows
Processed 170000 out of 272744 rows
Processed 180000 out of 272744 rows
Processed 190000 out of 272744 rows
Processed 200000 out of 272744 rows
Processed 210000 out of 272744 rows
Processed 220000 out of 272744 rows
Processed 230000 out of 272744 rows
Processed 240000 out of 272744 rows
Processed 250000 out of 272744 rows
Processed 260000 out of 272744 rows
Processed 270000 out of 272744 rows


## Remove isolated genes from network & gene expression

In [36]:
# kick out all genes without interactions (isolated nodes)
genes_with_interactions = adj_df.sum(axis=1) > 0
ppi_network = adj_df.ix[genes_with_interactions, genes_with_interactions]

# remove gene expression for isolated genes
gene_names = ppi_network.index
gene_expression_short = ge_scaled.loc[gene_expression['Name'].isin(gene_names)]
# build list of gene names and ids
gene_names_ids = np.stack([np.array(gene_expression_short.index),
                           np.array(gene_names)],
                          axis=1
                         )
gene_expression_short

,Pam3T16,Pam3T8,Pam3T16.1,Pam3T8.1,Pam3T16.2,Pam3T8.2,ControlT8,ControlT16,ControlT8.1,ControlT16.1,...,gfpmT8.1,gfpmT16.1,gfpmT8.2,gfpmT16.2,gfppT8,gfppT16,gfppT8.1,gfppT16.1,gfppT8.2,gfppT16.2
Ensembl-ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,-0.019636,-0.055894,-0.014149,-0.026348,-0.034298,-0.031775,-0.027954,-0.010898,0.011211,0.011268,...,-0.003864,0.022521,-0.018581,-0.039281,0.048819,0.030559,0.013568,0.014119,0.039539,0.028876
ENSG00000000938,-0.064162,0.016814,-0.009526,0.232816,0.042965,0.016543,-0.061241,-0.065906,0.103387,-0.027715,...,0.211692,0.148093,0.047569,-0.055339,-0.022985,-0.036726,0.014554,0.088354,-0.019915,-0.036164
ENSG00000001167,0.110516,0.042943,-0.004242,0.061898,0.081597,0.041112,0.049716,0.015254,0.058835,0.013988,...,0.008320,0.052419,0.040144,0.041011,0.072341,0.120034,0.053979,0.075344,0.075978,0.115994
ENSG00000001497,-0.012786,-0.027492,-0.051793,-0.070006,-0.067539,-0.043240,-0.074714,-0.060496,-0.062530,-0.046754,...,-0.058222,-0.073153,-0.069205,-0.075858,-0.058268,-0.056768,-0.044584,-0.060882,-0.069299,-0.065999
ENSG00000001626,-0.118962,-0.124057,-0.070946,-0.139674,-0.131325,-0.089921,-0.128608,-0.142557,-0.150866,-0.121095,...,-0.135072,-0.144909,-0.102280,-0.126709,-0.077457,-0.083253,-0.053783,-0.112923,-0.078409,-0.081514
ENSG00000001631,-0.000798,-0.002499,-0.022734,-0.030992,0.026794,0.016543,0.017222,0.009843,0.010443,-0.022276,...,-0.017922,-0.010794,-0.005756,0.047255,-0.037222,-0.040305,-0.016329,-0.023381,-0.031422,-0.023037
ENSG00000002016,-0.091562,-0.079751,-0.056416,-0.109020,-0.093592,-0.064533,-0.087395,-0.092960,-0.110923,-0.097523,...,-0.102270,-0.120990,-0.073930,-0.094593,-0.070029,-0.077526,-0.052798,-0.094555,-0.070258,-0.074353
ENSG00000002330,-0.107831,-0.110424,-0.063021,-0.127598,-0.108865,-0.082550,-0.105624,-0.115504,-0.128590,-0.098430,...,-0.108830,-0.131241,-0.084055,-0.109759,-0.073124,-0.069652,-0.050169,-0.090729,-0.069299,-0.070773
ENSG00000002746,-0.117250,-0.124057,-0.070946,-0.138745,-0.129529,-0.086645,-0.128608,-0.138048,-0.150866,-0.121095,...,-0.136009,-0.141492,-0.101605,-0.124033,-0.076838,-0.082537,-0.053455,-0.112157,-0.077450,-0.081514


## Get Labels

In [39]:
gene_names_ids

array([['ENSG00000000419', 'DPM1'],
       ['ENSG00000000938', 'FGR'],
       ['ENSG00000001167', 'NFYA'],
       ..., 
       ['ENSG00000278619', 'MRM1'],
       ['ENSG00000278817', 'DGCR6'],
       ['ENSG00000280987', 'MATR3']], dtype=object)

## Generate Training, Testing & Validation Splits

## Write to hdf5 File on Disk

In [38]:
# create gene names
string_dt = h5py.special_dtype(vlen=str)

f = h5py.File('../data/preprocessing/ppi_networks.h5', 'w')
f.create_dataset('consensusPathDB_ppi', data=ppi_network, shape=ppi_network.shape)
f.create_dataset('gene_expression', data=gene_expression_short, shape=gene_expression_short.shape)
f.create_dataset('gene_names', data=gene_names_ids, dtype=string_dt)

# doesn't exist yet :-(
#f.create_dataset('y_train', data=y_train, shape=y_train.shape)
#f.create_dataset('y_test', data=y_test, shape=y_test.shape)
#f.create_dataset('y_val', data=y_val, shape=y_val.shape)

#f.create_dataset('mask_train', data=mask_train, shape=mask_train.shape)
#f.create_dataset('mask_test', data=mask_test, shape=mask_test.shape)
#f.create_dataset('mask_val', data=mask_val, shape=mask_val.shape)
f.close()

In [25]:
f.close()

In [3]:
fname = '../data/preprocessing/ppi_networks.h5'
with h5py.File(fname, 'r') as f:
    gene_expression_data = f['gene_expression'][:]
    ppi_network = f['consensusPathDB_ppi'][:]
    gene_names = f['gene_names'][:]

In [4]:
gene_names

array(['DPM1', 'FGR', 'NFYA', ..., 'MRM1', 'DGCR6', 'MATR3'], dtype=object)

In [121]:
pr_val = np.array([3, 1, 2])
gene_names = np.array(['g1', 'g2', 'g3'])
sort_idx = pr_val.argsort()
gene_names_sorted = gene_names[sort_idx[::-1]]

In [122]:
gene_names_sorted

array(['g1', 'g3', 'g2'],
      dtype='<U2')

In [124]:
import networkx as nx

In [125]:
G = nx.DiGraph(nx.path_graph(4))
pr = nx.pagerank(G)

In [126]:
pr

{0: 0.17543839772251535,
 1: 0.32456160227748465,
 2: 0.32456160227748465,
 3: 0.17543839772251535}

In [128]:
import operator
pr_s = sorted(pr.items(), key=operator.itemgetter(1))[::-1]

In [131]:
gene_names_both

array([['TSPAN6', 'ENSG00000000003'],
       ['DPM1', 'ENSG00000000419'],
       ['SCYL3', 'ENSG00000000457'],
       ..., 
       ['CH17-132F21.5', 'ENSG00000281904'],
       ['RP11-439M15.1', 'ENSG00000281909'],
       ['LINC01144', 'ENSG00000281912']], dtype=object)

In [135]:
count = 1
for gene_idx, pr in pr_s:
    print ("{}\t{}\t{}\t{}\n".format(gene_names_both[gene_idx][1], gene_names_both[gene_idx][0], count, pr))
    count += 1

ENSG00000000457	SCYL3	1	0.32456160227748465

ENSG00000000419	DPM1	2	0.32456160227748465

ENSG00000000460	C1orf112	3	0.17543839772251535

ENSG00000000003	TSPAN6	4	0.17543839772251535



In [12]:
import tables

ModuleNotFoundError: No module named 'tables'